In [ ]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets
!pip install ipywebrtc
!pip install soundfile
!pip install pydub
!pip install ffmpeg-python

#역자 추가 코드
!pip install accelerate
!pip install bitsandbytes
!pip install sentencePiece
!pip install speechbrain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.8 MB/s eta 0:00:00
 

In [ ]:
import urllib.request

# audio_noisy.wav 파일 경로는 여러분의 알맞은 구글 디렉토리로 변경하세요.
filename = "/content/drive/MyDrive/Book6/Ch10/audio_noisy.wav"

In [ ]:
import torch
import torchaudio
from speechbrain.inference.enhancement import SpectralMaskEnhancement
from IPython.display import Audio

enhance_model = SpectralMaskEnhancement.from_hparams(
    source="speechbrain/metricgan-plus-voicebank",
    savedir="pretrained_models/metricgan-plus-voicebank",
)
waveform, sample_rate = torchaudio.load(filename)

# waveform이 스테레오 타입(2 채널)이면, 모노 타입(1 채널)로 변경
waveform = torch.mean(waveform, dim=0, keepdim=True)

# 통상적으로 SpeechBrain 사전 학습 모델은 16kHz의 오디오를 요구함
# 여러분의 오디오가 16kHz가 아니라면 다음 코드를 사용하여 변경
if sample_rate != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
    waveform = resampler(waveform)

# 이제 waveform 텐서가 음성 품질 개선 모델에 적합한 형태를 갖춤
# 다만 오디오 입력 전에 오디오 데이터 정규화 조치 필요
noisy = waveform / torch.max(torch.abs(waveform))

# 노이즈가 섞인 오디오 듣기
print("Noisy audio:")
display(Audio(noisy.squeeze().detach().numpy(), rate=16000))


hyperparams.yaml:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

enhance_model.ckpt:   0%|          | 0.00/7.59M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

Noisy audio:


In [ ]:
# lengths=torch.tensor([1.]) 코드로 오디오의 상대적 길이 텐서를 추가
enhanced = enhance_model.enhance_batch(noisy, lengths=torch.tensor([1.]))

# 개선된 시그널을 디스크에 저장
torchaudio.save('enhanced.wav', enhanced.cpu(), 16000)
# 개선된 오디오를 불러와 듣기
print("Enhanced audio:")
enhanced_audio = torchaudio.load('enhanced.wav')[0]
torchaudio.save('enhanced.wav', enhanced.cpu(), 16000)
display(Audio(enhanced_audio.detach().numpy(), rate=16000))

Enhanced audio:
